In [10]:
import importlib
import funciones_clusters as cl
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
from mne.channels import find_ch_adjacency
from mne.stats import spatio_temporal_cluster_test, ttest_ind_no_p, spatio_temporal_cluster_1samp_test
from functools import partial
%matplotlib qt5

In [14]:
importlib.reload(cl)

<module 'funciones_clusters' from 'c:\\Users\\jhquiza\\OneDrive - Universidad de Medellin\\JupyterNotebooks\\IAT\\funciones_clusters.py'>

In [3]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

evocados_co = {}
evocados_in = {}

for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

In [4]:
# Matrices de muestra de sujetos a analizar
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
negative = list(muestra[muestra['prejudice_level']=='negative']['subject'])
neutral = list(muestra[muestra['prejudice_level']=='neutral']['subject'])
positive = list(muestra[muestra['prejudice_level']=='positive']['subject'])
print(negative)

[23009, 22111, 23013, 21122, 22100, 22116, 22110, 21113, 22113, 21140, 23002, 21139, 23016, 23017, 23006, 21118, 22107, 21123, 22108]


In [6]:
for i in range(len(negative)):
    negative[i] = str(negative[i])
for i in range(len(neutral)):
    neutral[i] = str(neutral[i])
for i in range(len(positive)):
    positive[i] = str(positive[i])
__, __, negative_co, negative_in, __, __, __ = cl.matrices_cluster(negative, evocados_co, evocados_in)
__, __, neutral_co, neutral_in, __, __, __ = cl.matrices_cluster(neutral, evocados_co, evocados_in)
__, __, positive_co, positive_in, tiempos, canales, adjacency = cl.matrices_cluster(positive, evocados_co, evocados_in)

# Matrices de diferencias entre ensayos
negative_dif = negative_co - negative_in
neutral_dif = neutral_co - neutral_in
positive_dif = positive_co - positive_in
print(np.shape(positive_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(19, 256, 64)
(19, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(37, 256, 64)
(37, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(29, 256, 64)
(29, 256, 64)
(29, 256, 64)


In [15]:
canales = ['FZ', 'FC1', 'FC2']
tmin=0.0
tmax=0.796875
negative_co,neutral_co,positive_co,negative_in,neutral_in,positive_in,negative_dif,neutral_dif,positive_dif = cl.lista_erps_niveles(canales=canales, evocados_co=evocados_co, evocados_in=evocados_in, negative=negative, neutral=neutral, positive=positive, tmin=tmin, tmax=tmax)
negative_in

{'23009': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22111': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '23013': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21122': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22100': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22116': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22110': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21113': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '22113': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21140': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '23002': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, ~8 kB>,
 '21139': <Evoked | '' (average, N=1), 0 – 0.79688 sec, baseline off, 1 ch, 

In [16]:
# Diccionarios de toda la muestra
ROI_co = {**negative_co,**neutral_co,**positive_co}
ROI_in = {**negative_in,**neutral_in,**positive_in}
ROI_dif = {**negative_dif,**neutral_dif,**positive_dif}

In [15]:
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'both','legend.fontsize':20, 'legend.loc': 'lower left','xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)

In [16]:
def erps_grupos(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    negative_co,neutral_co,positive_co,negative_in,neutral_in,positive_in,negative_dif,neutral_dif,positive_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)

    # Diccionarios de toda la muestra
    ROI_co = {**negative_co,**neutral_co,**positive_co}
    ROI_in = {**negative_in,**neutral_in,**positive_in}
    ROI_dif = {**negative_dif,**neutral_dif,**positive_dif}

    # Listas de evocados de clústeres por grupos
    negative_co_list = list(negative_co.values())
    neutral_co_list = list(neutral_co.values())
    positive_co_list = list(positive_co.values())
    negative_in_list = list(negative_in.values())
    neutral_in_list = list(neutral_in.values())
    positive_in_list = list(positive_in.values())
    negative_dif_list = list(negative_dif.values())
    neutral_dif_list = list(neutral_dif.values())
    positive_dif_list = list(positive_dif.values())

    erps = {'negative_co':negative_co_list, 'neutral_co':neutral_co_list, 'positive_co':positive_co_list, 'negative_in':negative_in_list, 'neutral_in':neutral_in_list, 'positive_in':positive_in_list}

    # Gráfico de ensayos congruentes e incongruentes, con o sin intervalos de confianza
    figura_1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], linestyles=['solid', 'dashdot', 'dashed', 'solid', 'dashdot', 'dashed'], axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, split_legend=False, combine=None, show=False)

    # Gráfico de diferencias intervalos de confianza
    erps_dif = {'negative_dif':negative_dif_list, 'neutral_dif':neutral_dif_list, 'positive_dif':positive_dif_list}

    figura_2 = mne.viz.plot_compare_evokeds(evokeds=erps_dif, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff'], linestyles=['solid', 'dashdot', 'dashed'],axes=None, ci=0.95, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend=True, split_legend=False, combine=None, show=False)
    return figura_1, figura_2, ROI_co, ROI_in, ROI_dif

# Clústeres análisis univariante masivo

In [ ]:
medidas_clusteres = {}
clusteres = pd.read_csv('ROI NHST.csv', index_col='id')

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial (s)'][i]
    tmax = clusteres['t_final (s)'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

In [11]:
medidas_clusteres = {}
clusteres = pd.read_csv('clusters_0.05.csv', index_col='id')

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial (s)'][i]
    tmax = clusteres['t_final (s)'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

In [12]:
# Construcción dataframe de medidas de todos los clústeres
for k in medidas_clusteres.keys():
    num_cluster = str(k)
    medidas_clusteres[k].rename(columns={'mean_co':'mean_co_'+num_cluster,'mean_in':'mean_in_'+num_cluster,'dif_co_in':'dif_co_in_'+num_cluster}, inplace=True)

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],how='inner'),medidas_clusteres.values())
prueba = muestra.astype({'subject':'str'}).copy()
data = pd.merge(prueba, df_merged, on='subject', how='inner')
data.head()

,subject,type,exposure_level,group,victim_self_id,dscore_4,prejudice_level,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2,mean_co_3,mean_in_3,dif_co_in_3
0,24013,victim,high,victim,yes,-0.039706,neutral,-1.294701,-1.569333,0.274633,1.787661,1.464780,0.322881,3.097922,2.215356,0.882566
1,23004,excombatant,high,exparamilitar,yes,0.555094,positive,0.823093,-0.495528,1.318621,0.050548,0.342404,-0.291856,0.337562,0.230309,0.107253
2,24065,victim,high,victim,yes,0.236350,positive,-1.529942,-0.328146,-1.201796,0.157810,1.542331,-1.384521,-0.688134,0.430943,-1.119077
3,23009,excombatant,high,exparamilitar,yes,-0.439115,negative,0.323383,0.379465,-0.056082,-0.426418,-0.332914,-0.093505,0.223999,-0.470145,0.694144
4,21134,excombatant,high,exparamilitar,yes,-0.099490,neutral,-0.978303,-1.691090,0.712787,2.717402,-0.011120,2.728522,3.228046,0.288649,2.939398


In [13]:
data.to_csv('medidas_clusteres_0.05.csv', index=False)

# Análisis tradicional

In [18]:
# Gráficos clúster toda la época
canales = ['FZ']
figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)

<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.

In [19]:
# Gráficos clúster toda la época
canales = ['FC1']
figura_1, figura_2, __, __, __ = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)

<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.

In [20]:
# Gráficos clúster toda la época
canales = ['FC2']
figura_1, figura_2, __, __, __ = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)

<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
<ipython-input-2-705dceb4ead0>:8: RuntimeWarning: Less than 2 channels in group "ROI" when combining by method "functools.partial(<function mean at 0x0000015F57127040>, axis=0)".
  ROI_evocado = mne.channels.

In [21]:
# Gráficos clúster toda la época
canales = ['FC2', 'FC1', 'FZ']
figura_1, figura_2, __, __, __ = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)

In [22]:
medidas_clusteres = {}
clusteres = pd.read_csv('ROI NHST.csv', index_col='id')

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial (s)'][i]
    tmax = clusteres['t_final (s)'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

<ipython-input-6-7e64924141b4>:6: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  negative_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
<ipython-input-6-7e64924141b4>:7: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  negative_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
<ipython-input-6-7e64924141b4>:6: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  negative_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
<ipython-input-6-7e64924141b4>:7: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  negative_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
<ipython-input-6-7e64924141b4>:6: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 

In [23]:
# Construcción dataframe de medidas de todos los clústeres
for k in medidas_clusteres.keys():
    num_cluster = str(k)
    medidas_clusteres[k].rename(columns={'mean_co':'mean_co_'+num_cluster,'mean_in':'mean_in_'+num_cluster,'dif_co_in':'dif_co_in_'+num_cluster}, inplace=True)

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],how='inner'),medidas_clusteres.values())
prueba = muestra.astype({'subject':'str'}).copy()
data = pd.merge(prueba, df_merged, on='subject', how='inner')
data.head()

,subject,type,exposure_level,group,victim_self_id,dscore_4,prejudice_level,mean_co_1ci,mean_in_1ci,dif_co_in_1ci,...,dif_co_in_6d,mean_co_7d,mean_in_7d,dif_co_in_7d,mean_co_8d,mean_in_8d,dif_co_in_8d,mean_co_9d,mean_in_9d,dif_co_in_9d
0,24013,victim,high,victim,yes,-0.039706,neutral,0.547537,0.163316,0.384222,...,0.728155,-1.000159,-0.979226,-0.020933,-0.806365,0.557619,-1.363984,1.853521,1.163584,0.689937
1,23004,excombatant,high,exparamilitar,yes,0.555094,positive,0.870352,0.262133,0.608219,...,0.337136,-0.173078,-0.038214,-0.134864,-0.750159,-1.168147,0.417988,0.077534,0.329425,-0.251891
2,24065,victim,high,victim,yes,0.236350,positive,0.605925,0.170672,0.435253,...,0.130723,0.674061,1.572955,-0.898894,-0.482365,1.735003,-2.217369,0.763382,2.136326,-1.372944
3,23009,excombatant,high,exparamilitar,yes,-0.439115,negative,0.122963,0.386481,-0.263519,...,-0.401585,1.339968,1.356906,-0.016938,0.274615,-0.249776,0.524391,-0.098663,-0.177807,0.079144
4,21134,excombatant,high,exparamilitar,yes,-0.099490,neutral,0.125342,0.600217,-0.474875,...,-0.073078,0.364263,0.264410,0.099853,0.242256,-1.014046,1.256302,2.366287,-0.041778,2.408066


In [25]:
data.to_csv('medidas ROI NHST.csv', index=False)

In [24]:
muestra = pd.read_csv('Datos_EEG_demograficos_comportamentales_IAT.csv')
muestra = muestra.astype({'subject':'str'}).copy()
todas = pd.merge(muestra, data, left_on='subject', right_on='subject', how='inner')
todas.head()

,subject,co_cluster_1,in_cluster_1,dif_cluster_1,co_cluster_2,in_cluster_2,dif_cluster_2,co_cluster_3,in_cluster_3,dif_cluster_3,...,dif_co_in_6d,mean_co_7d,mean_in_7d,dif_co_in_7d,mean_co_8d,mean_in_8d,dif_co_in_8d,mean_co_9d,mean_in_9d,dif_co_in_9d
0,21100,1.056340,1.880976,-0.824636,0.889261,2.534708,-1.645447,0.301905,0.077673,0.224233,...,-1.780015,0.345875,2.335911,-1.990036,-1.130752,1.213659,-2.344411,-1.774957,1.214712,-2.989670
1,21101,0.988041,0.180916,0.807125,2.462776,1.612335,0.850441,1.449686,0.328569,1.121117,...,0.200957,1.950808,1.426324,0.524484,1.683822,1.161915,0.521906,1.367259,1.400992,-0.033733
2,21102,0.868476,-0.476409,0.563257,-1.502625,-2.084196,0.581571,0.764647,0.485184,0.716129,...,1.124414,-0.609665,-1.813374,1.203709,-2.131019,-3.063623,0.932604,-0.217983,-1.454689,1.236706
3,21103,-0.052317,1.288992,-1.341309,-0.628858,1.257010,-1.885868,-1.108922,0.751041,-1.859964,...,-1.128446,-0.677127,0.757168,-1.434296,-2.192741,-0.102213,-2.090528,-2.170242,-0.115299,-2.054943
4,21104,-0.476336,1.342512,-1.818847,0.998859,1.614534,-0.615675,-0.550279,1.271165,-1.821444,...,-0.894630,0.751566,1.638024,-0.886458,1.296737,1.463358,-0.166621,1.746829,2.251423,-0.504594


# Análisis clústeres hallados por métodos univariantes masivos

In [9]:
# Clúster 1
# Gráficos clúster toda la época
canales = ['O1', 'P9', 'PO7', 'PO3']
figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)

# Gráficos clúster en ventana de tiempo
tmin = 0.13671875
tmax = 0.1796875
figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)

# Medidas clúster
medidas_ROI_1 = pd.DataFrame(columns= ['subject','mean_co_1', 'mean_in_1', 'dif_co_in_1'])
for k in ROI_co.keys():
    mean_ROI_co = medidas(ROI_co[k], tmin, tmax)
    mean_ROI_in = medidas(ROI_in[k], tmin, tmax)
    mean_ROI_dif = medidas(ROI_dif[k], tmin, tmax)
    medidas_ROI_1 =  medidas_ROI_1.append({'subject': k, 'mean_co_1': mean_ROI_co[0], 'mean_in_1': mean_ROI_in[0], 'dif_co_in_1': mean_ROI_dif[0]}, ignore_index=True)
medidas_ROI_1.head()

,subject,mean_co_1,mean_in_1,dif_co_in_1
0,23009,0.737255,-0.741527,1.478782
1,22111,-2.141766,-1.233047,-0.908719
2,23013,-1.027724,0.032686,-1.060410
3,21122,-0.528043,-0.810878,0.282835
4,22100,0.189268,-0.660012,0.849281


## Procesamiento batch todos los clústeres

In [10]:
clusteres = pd.read_csv('Clústeres niveles nueva muestra sin ambos 0.5.csv', index_col='id')
clusteres.head()

,electrodos,t_inicial (s),t_final (s),ventana (ms)
id,,,,
1d,"FC2, FCZ, CZ, C2, FZ, F1",0.558594,0.796875,238.28125
2d,"AF4, F2, F4, FC4",0.136719,0.253906,117.18750
3d,"AF4, AFZ, FPZ",0.281250,0.460938,179.68750
4d,"P9, PO7",0.132812,0.226562,93.75000
5d,"CZ, C2, C1, CP1",0.382812,0.429688,46.87500


In [11]:
medidas_clusteres = {}

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial (s)'][i]
    tmax = clusteres['t_final (s)'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

C:\Users\jhquiza\AppData\Roaming\Python\Python38\site-packages\mne\viz\evoked.py:2283: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  axes = (plt.subplots(figsize=(8, 6))[1] for _ in ch_types)


In [12]:
# Construcción dataframe de medidas de todos los clústeres
for k in medidas_clusteres.keys():
    num_cluster = str(k)
    medidas_clusteres[k].rename(columns={'mean_co':'mean_co_'+num_cluster,'mean_in':'mean_in_'+num_cluster,'dif_co_in':'dif_co_in_'+num_cluster}, inplace=True)

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],how='inner'),medidas_clusteres.values())
prueba = muestra.astype({'subject':'str'}).copy()
data = pd.merge(prueba, df_merged, on='subject', how='inner')
data.to_csv('medidas_clusteres_nueva_muestra_sin_ambos_0.5.csv', index=False)

In [13]:
muestra = pd.read_csv('Datos_EEG_demograficos_comportamentales_IAT.csv')
muestra = muestra.astype({'subject':'str'}).copy()
todas = pd.merge(muestra, data, left_on='subject', right_on='subject', how='inner')
todas

,subject,co_cluster_1,in_cluster_1,dif_cluster_1,co_cluster_2,in_cluster_2,dif_cluster_2,co_cluster_3,in_cluster_3,dif_cluster_3,...,dif_co_in_13i,mean_co_14i,mean_in_14i,dif_co_in_14i,mean_co_15i,mean_in_15i,dif_co_in_15i,mean_co_16i,mean_in_16i,dif_co_in_16i
0,21100,1.056340,1.880976,-0.824636,0.889261,2.534708,-1.645447,0.301905,0.077673,0.224233,...,-0.607235,-0.892113,0.919095,-1.811208,-1.259528,1.528726,-2.788255,-2.175488,0.849422,-3.024910
1,21101,0.988041,0.180916,0.807125,2.462776,1.612335,0.850441,1.449686,0.328569,1.121117,...,1.195885,1.421793,0.525325,0.896468,0.047711,-0.093837,0.141548,-0.653986,-0.355987,-0.297999
2,21102,0.868476,-0.476409,0.563257,-1.502625,-2.084196,0.581571,0.764647,0.485184,0.716129,...,0.067355,4.502973,1.599339,2.903634,4.688957,1.637045,3.051913,4.990143,2.796252,2.193891
3,21103,-0.052317,1.288992,-1.341309,-0.628858,1.257010,-1.885868,-1.108922,0.751041,-1.859964,...,-1.358142,-1.350047,0.527549,-1.877596,-3.064519,-0.293128,-2.771391,-1.869640,-0.239761,-1.629879
4,21104,-0.476336,1.342512,-1.818847,0.998859,1.614534,-0.615675,-0.550279,1.271165,-1.821444,...,0.446969,-0.205608,1.343138,-1.548747,0.756659,1.933314,-1.176656,0.325341,1.365467,-1.040126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,24059,1.029033,0.592676,0.436357,0.666498,-0.141249,0.807748,-0.840527,-1.048173,0.207646,...,-1.230873,0.225280,0.311221,-0.085941,0.679759,0.119276,0.560483,-0.001587,0.126636,-0.128223
81,24065,-0.013765,-0.062560,0.048796,0.476309,2.214158,-1.737849,-1.966616,-0.269820,-1.696795,...,-1.854919,-0.565286,1.493918,-2.059205,0.494237,2.266143,-1.771906,0.681501,1.655508,-0.974007
82,24069,1.114822,1.381885,-0.267064,0.924062,0.951642,-0.027580,-0.034984,-0.209021,0.205522,...,-0.101261,0.655936,0.824935,-0.168999,1.134813,0.788861,0.345952,0.525793,-1.725839,2.251632
83,24073,1.073195,0.468144,0.605051,0.887057,1.228196,-0.341139,-0.278220,0.249028,-0.527248,...,1.994515,-1.103685,-0.008589,-1.095097,-0.385238,0.624037,-1.009275,1.369714,2.420293,-1.050579


In [14]:
todas.to_csv('medidas_clusteres_nueva_muestra_sin_ambos_0.5.csv', index=False)

In [113]:
def clusters_color(df):
    # Configuro tamaños de fuentes
    params = {'axes.labelsize':20, 'axes.titlesize': 20, 'legend.fontsize':20, 'legend.loc': 'upper left', 'xtick.labelsize': 14, 'ytick.labelsize': 14}
    plt.rcParams.update(params)
    df.rename(columns = {'Unnamed: 0': 'Time'}, inplace=True)
    df = df[df['Time']>= 0]
    df.set_index('Time', inplace=True)
    df = df.transpose()
    mask = df>0.2
    figura = sns.heatmap(df, vmin=0.0, vmax=0.2, cmap='hot', linewidths=0., linecolor='white', xticklabels=8, yticklabels=2, square=False, mask=mask)
    plt.show()
    return (figura)

In [168]:
df =  pd.read_csv('p_values_clusteres_niveles_incongruentes_nueva_muestra_sin_ambos.csv')
figura = clusters_color(df)

# Medidas y gráficas víctimas - no víctimas

In [8]:
# Matrices de muestra de sujetos a analizar
df.reset_index(inplace=True)
not_self_victim = list(df[df['victim_self']=='no']['subject'])
yes_self_victim = list(df[df['victim_self']=='yes']['subject'])
for i in range(len(not_self_victim)):
    not_self_victim [i] = str(not_self_victim[i])
for i in range(len(yes_self_victim)):
    yes_self_victim[i] = str(yes_self_victim[i])
__, __, not_self_victim_co, not_self_victim_in, __, __, __ = matrices_cluster(not_self_victim)
__, __, yes_self_victim_co, yes_self_victim_in, __, __, __ = matrices_cluster(yes_self_victim)

# Matrices de diferencias entre ensayos
not_self_victim_dif =  not_self_victim_co - not_self_victim_in
yes_self_victim_dif = yes_self_victim_co - yes_self_victim_in
print(np.shape(not_self_victim_dif))
print(np.shape(yes_self_victim_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(35, 256, 64)
(35, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(59, 256, 64)
(59, 256, 64)
(35, 256, 64)
(59, 256, 64)


In [16]:
def lista_evocados_grupos(canales, tmin=0.0, tmax=0.796875):
    yes_co = {}
    yes_in = {}
    yes_dif = {}
    for sujeto in yes_self_victim:
        yes_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        yes_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = yes_co[sujeto].data - yes_in[sujeto].data
        yes_dif[sujeto] = mne.EvokedArray(data=data_dif, info=yes_in[sujeto].info, tmin=yes_in[sujeto].tmin)
    not_co = {}
    not_in = {}
    not_dif = {}
    for sujeto in not_self_victim:
        not_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        not_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = not_co[sujeto].data - not_in[sujeto].data
        not_dif[sujeto] = mne.EvokedArray(data=data_dif, info=not_in[sujeto].info, tmin=not_in[sujeto].tmin)

    return yes_co, not_co, yes_in, not_in, yes_dif, not_dif

In [22]:
def erps_grupos(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    yes_co, not_co, yes_in, not_in, yes_dif,not_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)

    # Diccionarios de toda la muestra
    ROI_co = {**yes_co,**not_co}
    ROI_in = {**yes_in,**not_in}
    ROI_dif = {**yes_dif,**not_dif}

    # Listas de evocados de clústeres por grupos
    yes_self_victim_co_list = list(yes_co.values())
    not_self_victim_co_list = list(not_co.values())
    yes_self_victim_in_list = list(yes_in.values())
    not_self_victim_in_list = list(not_in.values())
    yes_self_victim_dif_list = list(yes_dif.values())
    not_self_victim_dif_list = list(not_dif.values())

    erps = {'yes_self_victim_co':yes_self_victim_co_list, 'not_self_victim_co':not_self_victim_co_list, 'yes_self_victim_in':yes_self_victim_in_list, 'not_self_victim_in':not_self_victim_in_list}

    # Gráfico de ensayos congruentes e incongruentes, con o sin intervalos de confianza
    figura_1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#d62728ff', '#9467bdff'], linestyles=['solid', 'dashdot', 'solid', 'dashdot'], axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, split_legend=False, combine=None, show=False)

    # Gráfico de diferencias intervalos de confianza
    erps_dif = {'yes_self_victim_dif':yes_self_victim_dif_list, 'not_self_victim_dif':not_self_victim_dif_list}

    figura_2 = mne.viz.plot_compare_evokeds(evokeds=erps_dif, colors=['#1f77b4ff', '#ff7f0eff'], linestyles=['solid', 'dashdot'],axes=None, ci=0.95, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend=True, split_legend=False, combine=None, show=False)
    return figura_1, figura_2, ROI_co, ROI_in, ROI_dif

In [23]:
clusteres = pd.read_csv('Clústeres grupos de actores diferencias sin ambos.csv', index_col='id')
clusteres.head()

,electrodos,t_inicial,t_final,ventana (ms),prueba
id,,,,,
1,"F1, FC1, F3, FC3",0.16016,0.36328,203.12,diferencias
2,"FZ, F2, FC2, FCZ, CZ",0.27344,0.33594,62.50,diferencias
3,"C1, C3, CP3, CP1, P1, P3",0.48437,0.53125,46.88,diferencias
4,"F3, AF3, F1",0.50781,0.60547,97.66,diferencias


In [24]:
medidas_clusteres = {}

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial'][i]
    tmax = clusteres['t_final'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

In [26]:
# Construcción dataframe de medidas de todos los clústeres
for k in medidas_clusteres.keys():
    num_cluster = str(k)
    medidas_clusteres[k].rename(columns={'mean_co':'mean_co_'+num_cluster,'mean_in':'mean_in_'+num_cluster,'dif_co_in':'dif_co_in_'+num_cluster}, inplace=True)

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],how='inner'),medidas_clusteres.values())
prueba = df.astype({'subject':'str'}).copy()
data = pd.merge(prueba, df_merged, on='subject', how='inner')
data.to_csv('medidas_clusteres_victims_self.csv', index=False)

# Medidas y gráficas civiles, víctimas, excombatientes no víctimas y excombatientes víctimas

In [28]:
df['new_classes'] = df['type']

,subject,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,...,AAP,TPA,TPV,EASPA,ANA,BN,ANC,AGA.1,TAPV,new_classes
0,21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant
1,21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,excombatant
2,21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant
3,21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant
4,21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,24059,victim,low,victim,NaN,no,44,11,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,victim
93,24065,victim,high,victim,yes,yes,49,6,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,victim
94,24069,victim,high,victim,no,no,22,11,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,victim
95,24073,victim,high,victim,yes,yes,70,11,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,victim


In [42]:
for i in range(len(df)):
    if (df['victim_self'][i]=='yes') & (df['type'][i]=='excombatant'):
        df['new_classes'][i] = 'excombatant victim'
    elif (df['victim_self'][i]=='no') & (df['type'][i]=='excombatant'):
        df['new_classes'][i] = 'excombatant not victim'
df.head()

<ipython-input-42-f37916054364>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_classes'][i] = 'excombatant victim'
<ipython-input-42-f37916054364>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_classes'][i] = 'excombatant not victim'


,subject,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,...,AAP,TPA,TPV,EASPA,ANA,BN,ANC,AGA.1,TAPV,new_classes
0,21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant victim
1,21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,excombatant victim
2,21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant victim
3,21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant victim
4,21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,excombatant victim


In [43]:
# Matrices de muestra de sujetos a analizar
civilian_non_victim = list(df[df['new_classes']=='control']['subject'])
civilian_victim = list(df[df['new_classes']=='victim']['subject'])
excombatant_non_victim = list(df[df['new_classes']=='excombatant not victim']['subject'])
excombatant_victim = list(df[df['new_classes']=='excombatant victim']['subject'])

for i in range(len(civilian_non_victim)):
    civilian_non_victim [i] = str(civilian_non_victim[i])
for i in range(len(civilian_victim)):
    civilian_victim [i] = str(civilian_victim[i])
for i in range(len(excombatant_non_victim)):
    excombatant_non_victim [i] = str(excombatant_non_victim[i])
for i in range(len(excombatant_victim)):
    excombatant_victim [i] = str(excombatant_victim[i])

__, __, civilian_non_victim_co, civilian_non_victim_in, __, __, __ = matrices_cluster(civilian_non_victim)
__, __, civilian_victim_co, civilian_victim_in, __, __, __ = matrices_cluster(civilian_victim)
__, __, excombatant_non_victim_co, excombatant_non_victim_in, __, __, __ = matrices_cluster(excombatant_non_victim)
__, __, excombatant_victim_co, excombatant_victim_in, __, __, __ = matrices_cluster(excombatant_victim)

# Matrices de diferencias entre ensayos
civilian_non_victim_dif = civilian_non_victim_co - civilian_non_victim_in
civilian_victim_dif = civilian_victim_co - civilian_victim_in
excombatant_non_victim_dif = excombatant_non_victim_co - excombatant_non_victim_in
excombatant_victim_dif = excombatant_victim_co - excombatant_victim_in

print(np.shape(civilian_non_victim_dif))
print(np.shape(civilian_victim_dif))
print(np.shape(excombatant_non_victim_dif))
print(np.shape(excombatant_victim_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(16, 256, 64)
(16, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 256, 64)
(23, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(13, 256, 64)
(13, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(45, 256, 64)
(45, 256, 64)
(16, 256, 64)
(23, 256, 64)
(13, 256, 64)
(45, 256, 64)


In [44]:
def lista_evocados_grupos(canales, tmin=0.0, tmax=0.796875):
    civil_no_co = {}
    civil_no_in = {}
    civil_no_dif = {}
    for sujeto in civilian_non_victim:
        civil_no_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        civil_no_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = civil_no_co[sujeto].data - civil_no_in[sujeto].data
        civil_no_dif[sujeto] = mne.EvokedArray(data=data_dif, info=civil_no_in[sujeto].info, tmin=civil_no_in[sujeto].tmin)

    civil_si_co = {}
    civil_si_in = {}
    civil_si_dif = {}
    for sujeto in civilian_victim:
        civil_si_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        civil_si_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = civil_si_co[sujeto].data - civil_si_in[sujeto].data
        civil_si_dif[sujeto] = mne.EvokedArray(data=data_dif, info=civil_si_in[sujeto].info, tmin=civil_si_in[sujeto].tmin)

    excombatant_no_co = {}
    excombatant_no_in = {}
    excombatant_no_dif = {}
    for sujeto in excombatant_non_victim:
        excombatant_no_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        excombatant_no_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = excombatant_no_co[sujeto].data - excombatant_no_in[sujeto].data
        excombatant_no_dif[sujeto] = mne.EvokedArray(data=data_dif, info=excombatant_no_in[sujeto].info, tmin=excombatant_no_in[sujeto].tmin)

    excombatant_si_co = {}
    excombatant_si_in = {}
    excombatant_si_dif = {}
    for sujeto in excombatant_victim:
        excombatant_si_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        excombatant_si_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = excombatant_si_co[sujeto].data - excombatant_si_in[sujeto].data
        excombatant_si_dif[sujeto] = mne.EvokedArray(data=data_dif, info=excombatant_si_in[sujeto].info, tmin=excombatant_si_in[sujeto].tmin)

    return civil_no_co, civil_si_co, excombatant_no_co, excombatant_si_co, civil_no_in, civil_si_in, excombatant_no_in, excombatant_si_in, civil_no_dif, civil_si_dif, excombatant_no_dif, excombatant_si_dif

In [49]:
def erps_grupos(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    civil_no_co, civil_si_co, excombatant_no_co, excombatant_si_co, civil_no_in, civil_si_in, excombatant_no_in, excombatant_si_in, civil_no_dif, civil_si_dif, excombatant_no_dif, excombatant_si_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)

    # Diccionarios de toda la muestra
    ROI_co = {**civil_no_co, **civil_si_co, **excombatant_no_co, **excombatant_si_co}
    ROI_in = {**civil_no_in, **civil_si_in, **excombatant_no_in, **excombatant_si_in}
    ROI_dif = {**civil_no_dif, **civil_si_dif, **excombatant_no_dif, **excombatant_si_dif}

    # Listas de evocados de clústeres por grupos
    civil_no_co_list = list(civil_no_co.values())
    civil_si_co_list = list(civil_si_co.values())
    excombatant_no_co_list = list(excombatant_no_co.values())
    excombatant_si_co_list = list(excombatant_si_co.values())
    civil_no_in_list = list(civil_no_in.values())
    civil_si_in_list = list(civil_si_in.values())
    excombatant_no_in_list = list(excombatant_no_in.values())
    excombatant_si_in_list = list(excombatant_si_in.values())
    civil_no_dif_list = list(civil_no_dif.values())
    civil_si_dif_list = list(civil_si_dif.values())
    excombatant_no_dif_list = list(excombatant_no_dif.values())
    excombatant_si_dif_list = list(excombatant_si_dif.values())

    erps = {'civil_no_co':civil_no_co_list, 'civil_si_co':civil_si_co_list, 'excombatant_no_co':excombatant_no_co_list, 'excombatant_si_co':excombatant_si_co_list, 'civil_no_in':civil_no_in_list, 'civil_si_in':civil_si_in_list, 'excombatant_no_in':excombatant_no_in_list, 'excombatant_si_in':excombatant_si_in_list}

    # Gráfico de ensayos congruentes e incongruentes, con o sin intervalos de confianza
    figura_1 = mne.viz.plot_compare_evokeds(evokeds=erps, linestyles=['solid','dashdot','dashed','dotted','solid','dashdot','dashed','dotted'], axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, split_legend=False, combine=None, show=False)

    # Gráfico de diferencias intervalos de confianza
    erps_dif = {'civil_no_dif':civil_no_dif_list, 'civil_si_dif':civil_si_dif_list, 'excombatant_no_dif':excombatant_no_dif_list, 'excombatant_si_dif':excombatant_si_dif_list}

    figura_2 = mne.viz.plot_compare_evokeds(evokeds=erps_dif, linestyles=['solid','dashdot','dashed','dotted'],axes=None, ci=0.95, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend=True, split_legend=False, combine=None, show=False)
    return figura_1, figura_2, ROI_co, ROI_in, ROI_dif

In [50]:
medidas_clusteres = {}

for i in clusteres.index:
    canales = clusteres['electrodos'][i].split(', ')
    tmin = clusteres['t_inicial'][i]
    tmax = clusteres['t_final'][i]
    figura_1, figura_2, ROI_co, ROI_in, ROI_dif = erps_grupos(canales=canales, tmin=0.0, tmax=0.796875)
    figura_1[0].set(tight_layout=True)
    figura_1[0].savefig('cluster_' + str(i) + '_both_trials'+'.png', dpi=600)
    figura_2[0].set(tight_layout=True)
    figura_2[0].savefig('cluster_' + str(i) + '_differences'+'.png', dpi=600)
    figura_3, figura_4, __, __, __ = erps_grupos(canales=canales, tmin=tmin, tmax=tmax)
    figura_3[0].set(tight_layout=True)
    figura_3[0].savefig('cluster_' + str(i) + '_both_trials_time_interval'+'.png', dpi=600)
    figura_4[0].set(tight_layout=True)
    figura_4[0].savefig('cluster_' + str(i) + '_differences_time_interval'+'.png', dpi=600)
    # Medidas clúster
    medidas_clusteres[i] = pd.DataFrame(columns= ['subject','mean_co', 'mean_in', 'dif_co_in'])
    for k in ROI_co.keys():
        mean_ROI_co = medidas(ROI_co[k], tmin=tmin, tmax=tmax)
        mean_ROI_in = medidas(ROI_in[k], tmin=tmin, tmax=tmax)
        mean_ROI_dif = medidas(ROI_dif[k], tmin=tmin, tmax=tmax)
        medidas_clusteres[i] =  medidas_clusteres[i].append({'subject': k, 'mean_co': mean_ROI_co[0], 'mean_in': mean_ROI_in[0], 'dif_co_in': mean_ROI_dif[0]}, ignore_index=True)

In [51]:
# Construcción dataframe de medidas de todos los clústeres
for k in medidas_clusteres.keys():
    num_cluster = str(k)
    medidas_clusteres[k].rename(columns={'mean_co':'mean_co_'+num_cluster,'mean_in':'mean_in_'+num_cluster,'dif_co_in':'dif_co_in_'+num_cluster}, inplace=True)

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],how='inner'),medidas_clusteres.values())
prueba = df.astype({'subject':'str'}).copy()
data = pd.merge(prueba, df_merged, on='subject', how='inner')
data.to_csv('medidas_clusteres_civiles_victimas_excombatientes.csv', index=False)